# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which will be used multiple times in this notebook.

Imports and set magics:

In [79]:
#Installing programs in Anaconda Promt. 
#(1): pip install pandas-datareader
#(2): pip install git+https://github.com/elben10/pydst

#Importing used packages.
import numpy as np
import pandas as pd
import pandas_datareader # install in (1).
import pydst # install in (2).
import matplotlib.pyplot as plt

#DATACLEANING#

#We use pydst to use an API to Denmark's statistics
Dst = pydst.Dst(lang='en')
Dst.get_subjects() #Used to see list of different datasets. 
tables = Dst.get_tables(subjects=['16']) #We choose the main dataset for "Money and credit markets".
tables[tables.id == 'MPK49'] #We choose the subdataset for "Pension funds".
Vars = Dst.get_variables(table_id='MPK49')

#To find the variables we need, we inspect the table that we have imported:
Vars.values


#After picking out values, we can get our data:
Data = Dst.get_data(table_id = 'MPK49', variables={'AKTPAS':['5180','5190','5200'], 'TID':['*'], 'TYPE':['*']})
Data.rename(columns={'AKTPAS':'Assets & Liabilities','TID':'Year','TYPE':'Type','INDHOLD':'Amount'},inplace=True)

Index = Data.set_index('Year')
Sort = Index[['Assets & Liabilities','Type','Amount']]

Corporate=Sort[Sort['Type']=='Corporate pension funds'].sort_values(['Year','Type'])
Working = Corporate[Corporate['Assets & Liabilities']=='Number of working members'].sort_values(['Year','Type']).rename(columns={'Assets & Liabilities':'Currently working', 'Amount':'Average pension for currently working in DKK'})
Retired = Corporate[Corporate['Assets & Liabilities']=='Number of retired members'].sort_values(['Year','Type']).rename(columns={'Assets & Liabilities':'Currently retired', 'Amount':'Average pension for currently retired in DKK'})

#Retired_noyear = Retired[['Assets & Liabilities', 'Number of retired members']]

#Concatenate the two tables:
Concatenated_table = pd.concat([Working, Retired], axis=1)
Concatenated_table






#Removing the gender nicer look:
Final_table = Concatenated_table[['Year','disposable_income_men','disposable_income_women']]

#DATACLEANING COMPLETE#

#APPLYING METHODS#

#Creates a function with provides the difference between the genders in %:
def f(x):
    """Gives the procentual difference between the genders"""
    return round((x['disposable_income_men']/x['disposable_income_women']-1)*100, 2)

#Applying the function to the end of the table:
Final_table['Difference in %']=Final_table.apply(f, axis=1)




,Pension for currently working,Type,Amount,Pension for currently retired,Type,Amount
Year,,,,,,
2000,Number of working members,Corporate pension funds,11898,Number of retired members,Corporate pension funds,7749
2001,Number of working members,Corporate pension funds,11404,Number of retired members,Corporate pension funds,7292
2002,Number of working members,Corporate pension funds,9830,Number of retired members,Corporate pension funds,6950
2003,Number of working members,Corporate pension funds,9912,Number of retired members,Corporate pension funds,6965
2004,Number of working members,Corporate pension funds,8624,Number of retired members,Corporate pension funds,7543
2005,Number of working members,Corporate pension funds,8172,Number of retired members,Corporate pension funds,7785
2006,Number of working members,Corporate pension funds,7760,Number of retired members,Corporate pension funds,7979
2007,Number of working members,Corporate pension funds,7356,Number of retired members,Corporate pension funds,7976
2008,Number of working members,Corporate pension funds,6794,Number of retired members,Corporate pension funds,7905


In [81]:
#Import the neccesary packages
import numpy as np
import pandas as pd
import pydst
import matplotlib.pyplot as plt

#DATACLEANING#

#We use pydst to use an API to Denmark's statistics
Dst = pydst.Dst(lang='en')
Dst.get_data(table_id='INDKP101')
Vars = Dst.get_variables(table_id='INDKP101')

#To find the variables we need, we inspect the table that we have imported:
Vars.values

#After picking out values, we can get our data:
Everything = Dst.get_data(table_id = 'INDKP101', variables={'OMRÅDE':['000','01','02','03','04','05','06','07','08','09','10','11'], 'KOEN':['M','K'], 'TID':['*'], 'ENHED':['116'], 'INDKOMSTTYPE':['100']}).rename(columns={'OMRÅDE':'Municipality'})

#Changing the index to Municipality:
New_index = Everything.set_index('Municipality')

#Picking out the neccesary variables:
Sortet = New_index[['TID','KOEN','INDHOLD']].rename(columns={'KOEN':'Gender', 'TID':'Year', 'INDHOLD':'disposable income'})

#Making a table for each gender:
Men = Sortet[Sortet['Gender']=='Men'].sort_values(['Municipality','Year']).rename(columns={'disposable income':'disposable_income_men'})

Women = Sortet[Sortet['Gender']=='Women'].sort_values(['Municipality', 'Year']).rename(columns={'disposable income':'disposable_income_women'})

Women_without_year = Women[['Gender', 'disposable_income_women']]

#Concatenate the two tables:
Concatenated_table = pd.concat([Men, Women_without_year], axis=1)

Concatenated_table






#Removing the gender nicer look:
Final_table = Concatenated_table[['Year','disposable_income_men','disposable_income_women']]

#DATACLEANING COMPLETE#

#APPLYING METHODS#

#Creates a function with provides the difference between the genders in %:
def f(x):
    """Gives the procentual difference between the genders"""
    return round((x['disposable_income_men']/x['disposable_income_women']-1)*100, 2)

#Applying the function to the end of the table:
Final_table['Difference in %']=Final_table.apply(f, axis=1)
Final_table



,Year,disposable_income_men,disposable_income_women,Difference in %
Municipality,,,,
All Denmark,1987,89143,67614,31.84
All Denmark,1988,92738,72330,28.22
All Denmark,1989,97448,77035,26.50
All Denmark,1990,100886,80440,25.42
All Denmark,1991,105575,84162,25.44
...,...,...,...,...
Province Østsjælland,2014,257221,211311,21.73
Province Østsjælland,2015,262510,215751,21.67
Province Østsjælland,2016,268279,219023,22.49


In [8]:
%load_ext autoreload
%autoreload 2
# You can load your python module as this:
import dataproject as dp
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


HTTPError: 400 Client Error: Can't find variable: al for url: https://api.statbank.dk/v1/data/MPK49/BULK?lang=en&delimiter=Semicolon&al=5180,5190,5200&TID=*&AKTPAS=5180&TYPE=5410&Tid=2000

# Read and clean data

## Employment data

**Read the employment data** in ``RAS200.xlsx`` and **clean it** removing and renaming columns:

In [6]:
# a. load
empl = pd.read_excel('RAS200.xlsx', skiprows=2)

# b. drop columns
drop_these = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
empl.drop(drop_these, axis=1, inplace=True)

# c. rename columns
empl.rename(columns = {'Unnamed: 4':'municipality'}, inplace=True)

The dataset now looks like this:

In [8]:
empl.head()

,municipality,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,All Denmark,76.0,72.8,72.1,71.8,71.3,71.3,71.6,72.1,72.8,73.6
1,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8,74.4
2,Province Byen København,73.4,71.0,70.5,70.5,70.4,70.6,70.9,71.6,72.3,73.0
3,Copenhagen,72.5,70.0,69.7,69.6,69.4,69.7,70.0,70.8,71.5,72.2
4,Frederiksberg,75.6,73.7,72.7,72.8,73.0,73.3,73.5,74.0,74.7,75.6


**Remove all rows which are not municipalities**:

In [10]:
empl = dataproject.only_keep_municipalities(empl)
empl.head()

NameError: name 'dataproject' is not defined

**Convert the dataset to long format**:

In [ ]:
# a. rename year columns
mydict = {str(i):f'employment{i}' for i in range(2008,2018)}
empl.rename(columns = mydict, inplace=True)

# b. convert to long
empl_long = pd.wide_to_long(empl, stubnames='employment', i='municipality', j='year').reset_index()

# c. show
empl_long.head()

## Income data

**Read the income data** in ``INDKP101.xlsx`` and **clean it**:

In [14]:
# a. load
inc = pd.read_excel('INDKP101.xlsx', skiprows=2)

# b. drop and rename columns
inc.drop([f'Unnamed: {i}' for i in range(3)], axis=1, inplace=True)
inc.rename(columns = {'Unnamed: 3':'municipality'}, inplace=True)

# c. drop rows with missing
inc.dropna(inplace=True)

# d. remove non-municipalities
inc = dataproject.only_keep_municipalities(inc)

# e. convert to long
inc.rename(columns = {str(i):f'income{i}' for i in range(1986,2018)}, inplace=True)
inc_long = pd.wide_to_long(inc, stubnames='income', i='municipality', j='year').reset_index()

# f. show
inc_long.head(5)

NameError: name 'dataproject' is not defined

> **Note:** The function ``dataproject.only_keep_municipalities()`` is used on both the employment and the income datasets.

## Explore data set

In order to be able to **explore the raw data**, we here provide an **interactive plot** to show, respectively, the employment and income level in each municipality

The **static plot** is:

In [ ]:
def plot_empl_inc(empl,inc,dataset,municipality): 
    
    if dataset == 'Employment':
        df = empl
        y = 'employment'
    else:
        df = inc
        y = 'income'
    
    I = df['municipality'] == municipality
    ax = df.loc[I,:].plot(x='year', y=y, style='-o')

The **interactive plot** is:

In [18]:
widgets.interact(plot_empl_inc, 
    
    empl = widgets.fixed(empl_long),
    inc = widgets.fixed(inc_long),
    dataset = widgets.Dropdown(description='Dataset', 
                               options=['Employment','Income']),
    municipality = widgets.Dropdown(description='Municipality', 
                                    options=empl_long.municipality.unique())
                 
); 

NameError: name 'widgets' is not defined

ADD SOMETHING HERE IF THE READER SHOULD KNOW THAT E.G. SOME MUNICIPALITY IS SPECIAL.

# Merge data sets

We now create a data set with **municpalities which are in both of our data sets**. We can illustrate this **merge** as:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('inc', 'empl'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

In [22]:
merged = pd.merge(empl_long, inc_long, how='inner',on=['municipality','year'])

print(f'Number of municipalities = {len(merged.municipality.unique())}')
print(f'Number of years          = {len(merged.year.unique())}')

NameError: name 'inc_long' is not defined

# Analysis

To get a quick overview of the data, we show some **summary statistics by year**:

In [24]:
merged.groupby('year').agg(['mean','std']).round(2)

NameError: name 'merged' is not defined

ADD FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.